## Importar datos

Para importar datos se necesita

- Fecha_inicial: Mes inicial de los datos a imprimir
- Fecha_fina: Mes final de los datos a imprimir
- Symbol: El nombre de la empresa o comodity a buscar (Ej: 'WTI' para petroleo Brent)
- interval: El intervalo de tiempo donde se quiere buscar


In [1]:
import numpy as np
import pandas as pd
import requests
import datetime as date
import json


def obtainTecnichalIndicator(indicator, mes, symbol, interval, fileExport):
    url = 'https://www.alphavantage.co/query?function='+indicator+'&symbol='+symbol+'&interval='+interval+'&time_period=10&series_type=open&month='+mes+'&apikey='+apikey
    r = requests.get(url)
    data = r.json()
#     print('hola:'+str(data))
    
    rawData2 = pd.DataFrame(data)
    # print(rawData2)
    rawData2.to_csv(fileExport, sep=',')
#     return rawData2[7:]
    
    rawData2=rawData2[7:]
    
    indicatorData=pd.DataFrame()
    indicatorData['Date & Hour']=pd.to_datetime(rawData2.index)
    indicatorData[indicator]=[d.get(indicator) for d in rawData2['Technical Analysis: {}'.format(indicator)]]
    return indicatorData

def obtainDataMonth(mes, symbol, interval):    
    ####Para obtener los valores del precio y volumen de la empresa
    url_PriceVolume = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol='+symbol+'&interval='+interval+'&outputsize=full&month='+mes+'&apikey='+apikey
    r = requests.get(url_PriceVolume)
    
    data = r.json()
    print(url_PriceVolume)
#     print('hola:'+str(data))
    rawData = pd.DataFrame(data)
    # print(rawData)
    rawData.to_csv(rutaExport+'{}-{}.csv'.format(symbol, mes), sep=',')

def fixData(rawData):
    rawData=rawData[6:]
    rawData['Time Series ({})'.format(interval)]=[json.loads(d.replace("\'", "\"")) for d in rawData['Time Series ({})'.format(interval)]]
    
    priceandvolume=pd.DataFrame()
    priceandvolume['Date & Hour']=pd.to_datetime(rawData.index)
    priceandvolume['Open']=[d.get('1. open') for d in rawData['Time Series ({})'.format(interval)]]
#     print(priceandvolume['Open'])
    priceandvolume['High']=[d.get('2. high') for d in rawData['Time Series ({})'.format(interval)]]
    priceandvolume['Low']=[d.get('3. low') for d in rawData['Time Series ({})'.format(interval)]]
    priceandvolume['Close']=[d.get('4. close') for d in rawData['Time Series ({})'.format(interval)]]
    priceandvolume['Volume']=[d.get('5. volume') for d in rawData['Time Series ({})'.format(interval)]]
    # print(priceandvolume)

    ####Para obtener los indicadores SMA, EMA, RSI de la empresa
#     SMA=obtainTecnichalIndicator('SMA', mes, symbol, interval, 'SMAdata.csv')
#     EMA=obtainTecnichalIndicator('EMA', mes, symbol, interval, 'EMAdata.csv')
#     RSI=obtainTecnichalIndicator('RSI', mes, symbol, interval, 'RSIdata.csv')

    #######Juntando toda la información
    dataAll=priceandvolume
#     dataAll=pd.merge(dataAll, SMA, how="outer", on='Date & Hour')
#     dataAll=pd.merge(dataAll, EMA, how="outer", on='Date & Hour')
#     dataAll=pd.merge(dataAll, RSI, how="outer", on='Date & Hour')
#     print(dataAll)
#     dataAll.to_csv(rutaExport+'{}-{}.csv'.format(symbol, mes), sep=';', index=False)
    
    return dataAll

Para importar datos se necesita

- Fecha_inicial: Mes inicial de los datos a imprimir
- Fecha_fina: Mes final de los datos a imprimir
- Symbol: El nombre de la empresa o comodity a buscar (Ej: 'WTI' para petroleo Brent)
- interval: El intervalo de tiempo donde se quiere buscar

In [ ]:
import os
    
mes_inicial='2023-08'
mes_final='2023-10'

symbol='AMZN'
interval='60min'

apikey='FUU1SOYI000781S9'
# apikey='JRT0SXRCUXDNAI29'
# apikey='SI924APD1QSRYWSR'

rutaExport='Data/{}/'.format(symbol)

if not os.path.exists(rutaExport):
        os.mkdir(rutaExport)

meses=np.arange(mes_inicial, mes_final, dtype='datetime64[M]')
# print(meses)
# print(type(meses))
# print(type(meses[0]))

meses=np.datetime_as_string(meses)
# print(meses)
# print(type(meses))
# print(type(meses[0]))

for mes in meses:
    print("obteniendo los datos para el mes: {}".format(mes))
    obtainDataMonth(mes, symbol, interval)

totalData=pd.DataFrame()
for mes in meses:
    dataMes=pd.read_csv(rutaExport+'{}-{}.csv'.format(symbol, mes), sep=',', index_col=0)
    dataMes=fixData(dataMes)
    totalData=pd.concat([totalData, dataMes])
    
totalData.to_csv('{}-{}-to-{}.csv'.format(symbol, mes_inicial, mes_final), sep=';', index=False)

print('-'*30)
print("Finalizo correctamente")
print('-'*30)

obteniendo los datos para el mes: 2023-08
